# Meta-analysis with METAL

METAT is a command line tool, takes in a script that documenting the input sumstat files, the field of each of those sumstat input file, and then the end of scipt.

METAL analysis is enssentially a weighted sum of Z score, therefore the input of snps from each chrm can be processed seperately. To make sure the same region was analysis, the input of this workflow shall be a list of path to the sumstat, join by " " to be analyzed at once.

Also, The scheme of weighting at the moment is set to be standard error, changing it to sample size would require the upstream input have a columns named sample size, which they dont have it yet.

At the moment, the AVERAGEFREQ, MINMAXFREQ, and GENOMICCONTROL are by default set to off, turning on each of the option require additional input from upstream file.

The output file name of the seconde step will by default contains a 1 linking the desinateted prefix and surfix, which lead to the f'{wd}/{name}1.METAL.txt' file name

In [ ]:
[global]
parameter: sumstat_list_path = path
parameter: variant_id = 'variant_id'
parameter: Sample_size = 'N'
parameter: ALLELE = 'alt ref'
parameter: FREQ   = 'EFFECT_ALLELE_FREQ'
parameter: Beta = 'beta'
parameter: SE = 'se'
parameter: PVAL   = 'pval'
parameter: wd = path
parameter: SCHEME = 'STDERR'
parameter: AVERAGEFREQ = "OFF"
parameter: MINMAXFREQ  = "OFF"
parameter: GENOMICCONTROL  = "OFF"
parameter: container = str
import pandas as pd

sumstat_list = pd.read_csv(sumstat_list_path, sep = "\t")
sumstat_inv = sumstat_list.values.tolist()
name = "-".join(sumstat_list.columns.values[1:len(sumstat_list.columns.values)].tolist())

### Step.1 Script generation
The METAL script contains three sections, the first is the analysis parameters, the second is the inputs files, and the third is the ending of script.

In [ ]:
[METAL_1,generate_METAL_script]
input:  for_each = "sumstat_inv"
output: METAL_script = f'{wd:a}/{name}.{_sumstat_inv[0]}.METAL_script.txt'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '55G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout', container = container
        echo '
        # Meta-analysis weighted by:
        SCHEME   $[SCHEME]
        
        # Whether do genomics control
        GENOMICCONTROL $[GENOMICCONTROL]
        
        # Whether do AVERAGEFREQ or MINMAXFREQ control
        AVERAGEFREQ $[AVERAGEFREQ]
        MINMAXFREQ $[MINMAXFREQ] 
        OUTFILE $[wd:a]/$[name].$[_sumstat_inv[0]]. .METAL.txt' > $[_output]

        for i in $[" ".join(_sumstat_inv[1:len(_sumstat_inv)])] ; do 
        echo "     
        MARKER   $[variant_id]
        WEIGHT   $[Sample_size]
        ALLELE   $[ALLELE]
        #FREQ     $[FREQ]
        EFFECT   $[Beta]
        STDERR   $[SE]
        PVAL     $[PVAL]
        PROCESS  $i " >> $[_output];
        done 
        
        echo '
        ANALYZE' >> $[_output]

## Step 2. Metal analysis

In [ ]:
[METAL_2,Excute_METAL_script]
input: group_with = "sumstat_inv"
output: f'{wd}/{name}.{_sumstat_inv[0]}.1.METAL.txt'
task: trunk_workers = 1, trunk_size = 1, walltime = '24h', mem = '55G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container
       metal $[_input]

## Step 3. Reformat the output so that they are in the same formmat as the non metaled one

In [ ]:
[METAL_3,Output_reformatting]
input: group_with = "sumstat_inv"
output: f'{wd:a}/{name}.{_sumstat_inv[0]}.METAL.vcf.bgz',
        sumstat = f'{wd:a}/{name}.{_sumstat_inv[0]}.METAL.txt'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '55G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout' , container = container 
        import pandas as pd
        df = pd.read_csv($[_input:r],sep = "\t")
        df.columns = ["variant_id","alt","ref","beta","se","pval","Direction"]
        df = df.assign(
        pos = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[0].split(":")[1])).assign(
        chrom = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split(":")[0]))
        df.to_csv("$[_output[1]]",sep = "\t",index = 0)

R: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout' , container = container 
    library("dplyr")
    library("readr")
    library("purrr")
    ## Define a wrapper, modified from the gwasvcf packages, to create the vcf of needed.
        create_vcf = function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL, 
        se = NULL, pval = NULL, n = NULL, ncase = NULL, name = NULL) 
    {
        stopifnot(length(chrom) == length(pos))
        if (is.null(snp)) {
            snp <- paste0(chrom, ":", pos)
        }
        snp <- paste0(chrom, ":", pos)
        nsnp <- length(chrom)
        gen <- list()
        ## Setupt data content for each sample column
        if (!is.null(ea_af)) 
            gen[["AF"]] <- matrix(ea_af, nsnp)
        if (!is.null(effect)) 
            gen[["ES"]] <- matrix(effect, nsnp)
        if (!is.null(se)) 
            gen[["SE"]] <- matrix(se, nsnp)
        if (!is.null(pval)) 
            gen[["LP"]] <- matrix(-log10(pval), nsnp)
        if (!is.null(n)) 
            gen[["SS"]] <- matrix(n, nsnp)
        if (!is.null(ncase)) 
            gen[["NC"]] <- matrix(ncase, nsnp)
        gen <- S4Vectors::SimpleList(gen)
        
      ## Setup snps info for the fix columns
        gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
            end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
      ## Setup meta informations
         coldata <- S4Vectors::DataFrame(Studies = name, row.names = name)
        hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
            row.names = "fileformat")), sample = name)
        VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
            "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
            "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
            "Standard error of effect size estimate", "-log10 p-value for effect estimate", 
            "Alternate allele frequency in the association study", 
            "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"), 
            row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))
      ## Save only the meta information in the sample columns 
        VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
            rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
      ## Save VCF values
        vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
            exptData = list(header = hdr), geno = gen)
        VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
        VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
      ## Write fixed values
        VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
            return(sort(vcf))
        }
  
        input = read_delim($[_output[1]:r],"\t")
        input_effect = input$beta
        input_se = input$se
        df = tibble(snps = input$variant_id)
        df = df%>%mutate(    chr = map_dbl(snps,~read.table(text = .x,sep = ":",as.is = T)$V1),
                     pos_alt_ref = map_chr(snps,~read.table(text = .x,sep = ":",as.is = TRUE)$V2),
                     pos = map_dbl(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE)$V1),
                     alt = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V3),
                     ref = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V2))
        vcf = create_vcf(
           chrom = df$chr,
            pos = df$pos,
            ea = df$alt,
            nea = df$ref,
            effect = input_effect ,
            se =  input_se,
            name = "$[name]")
      VariantAnnotation::writeVcf(vcf,$[_output[0]:nr],index = TRUE)

In [ ]:
[METAL_4,recipe]
input: output_from("METAL_3")["sumstat"],group_by = "all"
output: f'{wd}/{name}.METAL_list.txt'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout' , container = container 
        import pandas as pd
        df = pd.DataFrame({"#chr" : pd.read_csv($[sumstat_list_path:r],sep = "\t")["#chr"].values.tolist()  ,  "$[name]"  : [$[_input:r,]] })
        df.to_csv("$[_output]",sep = "\t",index = 0)